In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.memory import ConversationBufferMemory

llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.1,
)

memory = ConversationBufferMemory(
    return_messages=True,
)

cache_dir = LocalFileStore("./.cache_day4/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator='\n\n',
    chunk_size=600,
    chunk_overlap=100,
    
)

loader = UnstructuredFileLoader("./files/1984_chapter3.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, 
    cache_dir,
)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages([
    ("system", 
     """
     You are a helpful assistant.
     Answer questions using only the following context AND the previous conversation history.
     If the answer cannot be found in either the context or the conversation history,
     just say you don't know. Do not make up an answer.
     
     {context}
     """),
     MessagesPlaceholder(variable_name="history"),
    ("human", "{question}"),
])

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = (
    {"context": retriever, "question": RunnablePassthrough()} 
    | RunnablePassthrough.assign(history=load_memory)
    | prompt 
    | llm
)

def invoke_chain(question):
    result = chain.invoke("Is aaronson guilty?")

    memory.save_context({"input": question}, {"output": result.content})

    print(result)



In [2]:
invoke_chain("Is aaronson guilty?")

content='According to the text, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.'


In [3]:
chain.invoke("What message did he write in the table?")

AIMessage(content='Winston traced "2+2=5" in the dust on the table almost unconsciously.')

In [4]:
chain.invoke("Who is Julia?")

AIMessage(content='Julia is a character in the text who is significant to Winston. She is someone he cared for deeply and had a romantic relationship with, but their connection was tested under the oppressive regime they lived in.')